<a href="https://colab.research.google.com/github/duper203/RAG_Techniques_with_upstage/blob/main/upstage/20_retrieval_with_feedback_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG System with Feedback Loop: Enhancing Retrieval and Response Quality






## Key Components

1. PDF Content Extraction: Extracts text from PDF documents
2. Vectorstore: Stores and indexes document embeddings for efficient retrieval
3. Retriever: Fetches relevant documents based on user queries
4. Language Model: Generates responses using retrieved documents
5. Feedback Collection: Gathers user feedback on response quality and relevance
6. Feedback Storage: Persists user feedback for future use
7. Relevance Score Adjustment: Modifies document relevance based on feedback
8. Index Fine-tuning: Periodically updates the vectorstore using accumulated feedback



## Method Details

1. Initial Setup
2. Query Processing
3. Feedback Collection
4. Relevance Score Adjustment
5. Retriever Update
6. Periodic Index Fine-tuning

In [17]:
! pip3 install -qU langchain-upstage langchain langchain-community faiss-cpu PyMuPDF

In [18]:
import os
import asyncio
from google.colab import userdata

from langchain_upstage import ChatUpstage, UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

import json
from typing import List, Dict, Any

os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Define document(s) path & Read PDf to string

In [19]:
path = "data/Understanding_Climate_Change.pdf"

In [20]:
def read_pdf_to_string(path):
    """
    Read a PDF document from the specified path and return its content as a string.

    Args:
        path (str): The file path to the PDF document.

    Returns:
        str: The concatenated text content of all pages in the PDF document.

    The function uses the 'fitz' library (PyMuPDF) to open the PDF document, iterate over each page,
    extract the text content from each page, and append it to a single string.
    """
    # Open the PDF document located at the specified path
    doc = fitz.open(path)
    content = ""
    # Iterate over each page in the document
    for page_num in range(len(doc)):
        # Get the current page
        page = doc[page_num]
        # Extract the text content from the current page and append it to the content string
        content += page.get_text()
    return content

In [21]:
import fitz
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
def encode_from_string(content, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a string into a vector store using OpenAI embeddings.

    Args:
        content (str): The text content to be encoded.
        chunk_size (int): The size of each chunk of text.
        chunk_overlap (int): The overlap between chunks.

    Returns:
        FAISS: A vector store containing the encoded content.

    Raises:
        ValueError: If the input content is not valid.
        RuntimeError: If there is an error during the encoding process.
    """

    if not isinstance(content, str) or not content.strip():
        raise ValueError("Content must be a non-empty string.")

    if not isinstance(chunk_size, int) or chunk_size <= 0:
        raise ValueError("chunk_size must be a positive integer.")

    if not isinstance(chunk_overlap, int) or chunk_overlap < 0:
        raise ValueError("chunk_overlap must be a non-negative integer.")

    try:
        # Split the content into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
        chunks = text_splitter.create_documents([content])

        # Assign metadata to each chunk
        for chunk in chunks:
            chunk.metadata['relevance_score'] = 1.0

        # Generate embeddings and create the vector store
        embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
        vectorstore = FAISS.from_documents(chunks, embeddings)

    except Exception as e:
        raise RuntimeError(f"An error occurred during the encoding process: {str(e)}")

    return vectorstore


In [22]:
content = read_pdf_to_string(path)
vectorstore = encode_from_string(content)
retriever = vectorstore.as_retriever()

In [32]:
llm = ChatUpstage(model="solar-pro")
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

In [33]:
def get_user_feedback(query, response, relevance, quality, comments=""):
    return {
        "query": query,
        "response": response,
        "relevance": int(relevance),
        "quality": int(quality),
        "comments": comments
    }

In [34]:
def store_feedback(feedback):
    with open("data/feedback_data.json", "a") as f:
        json.dump(feedback, f)
        f.write("\n")

In [35]:
def load_feedback_data():
    feedback_data = []
    try:
        with open("data/feedback_data.json", "r") as f:
            for line in f:
                feedback_data.append(json.loads(line.strip()))
    except FileNotFoundError:
        print("No feedback data file found. Starting with empty feedback.")
    return feedback_data

In [36]:
import json
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
class Response(BaseModel):
    answer: str = Field(..., title="The answer to the question. The options can be only 'Yes' or 'No'")

def adjust_relevance_scores(query: str, docs: List[Any], feedback_data: List[Dict[str, Any]]) -> List[Any]:
    # Create a prompt template for relevance checking
    relevance_prompt = PromptTemplate(
        input_variables=["query", "feedback_query", "doc_content", "feedback_response"],
        template="""
        Determine if the following feedback response is relevant to the current query and document content.
        You are also provided with the Feedback original query that was used to generate the feedback response.
        Current query: {query}
        Feedback query: {feedback_query}
        Document content: {doc_content}
        Feedback response: {feedback_response}

        Is this feedback relevant? Respond with only 'Yes' or 'No'.
        """
    )
    llm = ChatUpstage()

    # Create an LLMChain for relevance checking
    relevance_chain = relevance_prompt | llm.with_structured_output(Response)

    for doc in docs:
        relevant_feedback = []

        for feedback in feedback_data:
            # Use LLM to check relevance
            input_data = {
                "query": query,
                "feedback_query": feedback['query'],
                "doc_content": doc.page_content[:1000],
                "feedback_response": feedback['response']
            }
            # result = relevance_chain.invoke(input_data).answer

            # if result == 'yes':
            #     relevant_feedback.append(feedback)
            result = relevance_chain.invoke(input_data)

            if result is not None and result.answer.lower() == 'yes':  # Check if the result is valid
                relevant_feedback.append(feedback)


        # Adjust the relevance score based on feedback
        if relevant_feedback:
            avg_relevance = sum(f['relevance'] for f in relevant_feedback) / len(relevant_feedback)
            doc.metadata['relevance_score'] *= (avg_relevance / 3)  # Assuming a 1-5 scale, 3 is neutral

    # Re-rank documents based on adjusted scores
    return sorted(docs, key=lambda x: x.metadata['relevance_score'], reverse=True)

In [37]:
def fine_tune_index(feedback_data: List[Dict[str, Any]], texts: List[str]) -> Any:
    # Filter high-quality responses
    good_responses = [f for f in feedback_data if f['relevance'] >= 4 and f['quality'] >= 4]

    # Extract queries and responses, and create new documents
    additional_texts = []
    for f in good_responses:
        combined_text = f['query'] + " " + f['response']
        additional_texts.append(combined_text)

    # make the list a string
    additional_texts = " ".join(additional_texts)

    # Create a new index with original and high-quality texts
    all_texts = texts + additional_texts
    new_vectorstore = encode_from_string(all_texts)

    return new_vectorstore


In [38]:
query = "What is the greenhouse effect?"

# Get response from RAG system
response = qa_chain(query)

In [41]:
response

'The greenhouse effect is a natural process that occurs when certain gases in the Earth\'s atmosphere, such as carbon dioxide, methane, and nitrous oxide, trap heat from the sun, creating a "greenhouse" effect. This effect is essential for life on Earth, as it keeps the planet warm enough to support life. However, human activities have intensified this natural process, leading to a warmer climate.'

In [39]:
query = "What is the greenhouse effect?"

# Get response from RAG system
response = qa_chain(query)["result"]

relevance = 5
quality = 5

# Collect feedback
feedback = get_user_feedback(query, response, relevance, quality)

# Store feedback
store_feedback(feedback)

# Adjust relevance scores for future retrievals
docs = retriever.get_relevant_documents(query)
adjusted_docs = adjust_relevance_scores(query, docs, load_feedback_data())

# Update the retriever with adjusted docs
retriever.search_kwargs['k'] = len(adjusted_docs)
retriever.search_kwargs['docs'] = adjusted_docs

In [40]:
# Periodically (e.g., daily or weekly), fine-tune the index
new_vectorstore = fine_tune_index(load_feedback_data(), content)
retriever = new_vectorstore.as_retriever()